# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.dropna(inplace=True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configuring gmaps 
gmaps.configure(api_key=g_key)

In [5]:
#storing latitude and longitude as my locations
locations = cities_df[["Lat", "Lng"]]

#storing the humidity and my weights
humidity = cities_df["Humidity"].astype(float)

In [6]:
#plotting the heat map 
fig = gmaps.figure()

#creating the heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=cities_df['Humidity'].max())
                              
#adding the layer to the map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = cities_df[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"]  > 75) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 10)]
hotel_df = hotel_df.copy()
hotel_df['Hotel_Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
34,Nampula,-15.1165,39.2666,77.00,83,0,3.44,MZ,1612310548,
56,Comodoro Rivadavia,-45.8667,-67.5000,78.80,31,0,9.22,AR,1612310550,
117,Vila Velha,-20.3297,-40.2925,79.00,78,0,9.22,BR,1612310559,
148,Coyhaique,-45.5752,-72.0662,75.20,43,0,3.44,CL,1612310563,
359,Salalah,17.0151,54.0924,75.20,12,0,6.91,OM,1612310592,
401,Maceió,-9.6658,-35.7353,75.20,94,0,3.44,BR,1612310501,
478,Rawson,-43.3002,-65.1023,75.76,32,0,6.31,AR,1612310562,
561,Oussouye,12.4850,-16.5469,75.20,94,0,2.30,SN,1612310617,


In [11]:
param = {
    "location" : "-15.1165,39.2666",
    "type": "lodging",
    "key": g_key,
    "radius": 5000 }

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=param).json()

for i, row, in hotel_df.iterrows():
    param['location'] = str(hotel_df.loc[i,'Lat']) + ',' + str(hotel_df.loc[i,'Lng'])
    response = requests.get(base_url, params=param).json()
    hotel_df.loc[i,'Hotel_Name'] = response['results'][0]['name']  

In [13]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# Create bank symbol layer
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))